In [74]:
!pip install selenium
!pip install webdriver-manager
!pip install bs4

You should consider upgrading via the 'C:\Users\thoma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\thoma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\thoma\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [75]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time 
import pandas as pd

In [76]:
start = time.time()
# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install())

# si on a télécharger le driver directement dans le dossier 
# service = Service(executable_path="./chromedriver.exe")
# driver = webdriver.Chrome(service=service)

# Navigate to the website
#driver.get("https://www.billboard.com/charts/hot-100/")
driver.get("https://www.billboard.com/charts/billboard-global-200/")
#driver.get("https://www.billboard.com/charts/billboard-200/")

# Wait for the page to load
# driver.implicitly_wait(10)
time.sleep(10)

# reject all cookies
driver.find_element(By.XPATH,"/html/body/div[6]/div[2]/div/div/div[2]/div/div/button[1]").click()

iterateur100 = list(range(1,110,11))
iterateur100.remove(1)
iterateur200 = list(range(1,220,11))
iterateur200.remove(1)

# nb_max 111 ou 221
def scraper(list,xpath,iterateur,nb_max):
    for i in range(2,nb_max):
        if i not in iterateur : 
            WebElement = driver.find_element(By.XPATH,xpath%(i))
            list.append(WebElement.text)

    return list

XpathTitle = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/h3"
XpathArtist = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[1]/span"
XpathRank = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[1]/span "
XpathLastWeek = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[4]/span"
XpathPeakPosition = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[5]/span"
XpathWeeksOnChart = "/html/body/div[4]/main/div[2]/div[3]/div/div/div/div[2]/div[%d]/ul/li[4]/ul/li[6]/span"

allXpath = [XpathTitle,XpathArtist,XpathRank,XpathLastWeek,XpathPeakPosition,XpathWeeksOnChart]

title,artist,rank,last_week,peak_pos,weeks_on_chart = [],[],[],[],[],[]

allList = [title,artist,rank,last_week,peak_pos,weeks_on_chart]

i = [scraper(i,j,iterateur200,221) for i, j in zip(allList, allXpath)]

# for i, j in zip(allList, allXpath):
#     i = scraper(i,j,iterateur200,221)

df = pd.DataFrame(list(zip(title,artist,rank,last_week,peak_pos,weeks_on_chart)),columns=['Title','Artist','Rank','Last Week','Peak Positon','Weeks on charts'])
print(df)

# Close the webdriver
driver.close()

end = time.time()
temps = end - start
print("temps d'execution :",temps,"s")

C:\Users\thoma\AppData\Local\Temp\ipykernel_21568\3835546797.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


                            Title                    Artist Rank Last Week  \
0                         Flowers               Miley Cyrus    1         1   
1    Bzrp Music Sessions, Vol. 53        Bizarrap & Shakira    2         2   
2                       Kill Bill                       SZA    3         3   
3                          Unholy    Sam Smith & Kim Petras    4         5   
4                       Calm Down       Rema & Selena Gomez    5         4   
..                            ...                       ...  ...       ...   
195    Growin' Up And Gettin' Old                Luke Combs  196         -   
196                   Besos Moja2  Wisin & Yandel + Rosalia  197       185   
197             Murder In My Mind                  Kordhell  198       182   
198                  Vista Al Mar                   Quevedo  199       136   
199             Super Freaky Girl               Nicki Minaj  200       149   

    Peak Positon Weeks on charts  
0              1            

In [77]:
song_artists=list(zip( df['Artist'], df['Title'],))
print(song_artists)

[('Miley Cyrus', 'Flowers'), ('Bizarrap & Shakira', 'Bzrp Music Sessions, Vol. 53'), ('SZA', 'Kill Bill'), ('Sam Smith & Kim Petras', 'Unholy'), ('Rema & Selena Gomez', 'Calm Down'), ('Metro Boomin, The Weeknd & 21 Savage', "Creepin'"), ('RAYE Featuring 070 Shake', 'Escapism'), ('Taylor Swift', 'Anti-Hero'), ('David Guetta & Bebe Rexha', "I'm Good (Blue)"), ('Harry Styles', 'As It Was'), ('NewJeans', 'OMG'), ('NewJeans', 'Ditto'), ('Miguel', 'Sure Thing'), ('Meghan Trainor', 'Made You Look'), ('Stephen Sanchez', 'Until I Found You'), ('Yandel & Feid', 'Yandel 150'), ('Manuel Turizo', 'La Bachata'), ('JVKE', 'Golden Hour'), ('Tom Odell', 'Another Love'), ('OneRepublic', "I Ain't Worried"), ('The Weeknd & Ariana Grande', 'Save Your Tears'), ('Arcangel & Bad Bunny', 'La Jumpa'), ('The Weeknd', 'Die For You'), ('Fuerza Regida X Grupo Frontera', 'Bebe Dame'), ('Drake & 21 Savage', 'Rich Flex'), ('Bizarrap & Quevedo', 'Bzrp Music Sessions, Vol. 52'), ('Oliver Tree & Robin Schulz', 'Miss You'

In [93]:

# Initialize the webdriver
# de manière basique
# driver = webdriver.Chrome()

# avec ChromeDriverManager mieux car update du driver automatiquement
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get('https://genius.com/{}'.format(str("Sza-sos-lyrics")))
# driver.find_element(By.XPATH,"/html/body/div[7]/div[3]/div[1]/div/div[2]/div/button[1]").click()
# date = driver.find_element(By.XPATH,"/html/body/div[1]/main/div[1]/div[4]/div/div[1]/div[2]/div").text
# print(date)

# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
driver_genius = webdriver.Chrome(ChromeDriverManager().install())

# for song in song_artists:

    # ouvrir le site et accepter les cookies
driver_genius.get("https://genius.com/")
driver_genius.find_element(By.XPATH,"/html/body/div[2]/div[3]/div[1]/div/div[2]/div/button[1]").click()

# activer la recherche

element = driver_genius.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/form/input")
element.send_keys('Flowers', Keys.ARROW_DOWN)
element.submit()
driver_genius.find_element(By.XPATH,"/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[1]/search-result-section/div/div[2]/search-result-items/div/search-result-item/div/mini-song-card/a/div[2]").click()
driver_genius.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[3]/div[1]/div[7]/div[2]/button").click()
print(driver_genius.find_element(By.XPATH, "/html/body/div[1]/main/div[3]/div[3]/div[1]/div[7]/div[1]/div/div[5]/a[1]").text)

time.sleep(150)


C:\Users\thoma\AppData\Local\Temp\ipykernel_21568\2369115226.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver_genius = webdriver.Chrome(ChromeDriverManager().install())


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[3]/div[1]/div/div[2]/div/button[1]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00B16643]
	(No symbol) [0x00AABE21]
	(No symbol) [0x009ADA9D]
	(No symbol) [0x009E1342]
	(No symbol) [0x009E147B]
	(No symbol) [0x00A18DC2]
	(No symbol) [0x009FFDC4]
	(No symbol) [0x00A16B09]
	(No symbol) [0x009FFB76]
	(No symbol) [0x009D49C1]
	(No symbol) [0x009D5E5D]
	GetHandleVerifier [0x00D8A142+2497106]
	GetHandleVerifier [0x00DB85D3+2686691]
	GetHandleVerifier [0x00DBBB9C+2700460]
	GetHandleVerifier [0x00BC3B10+635936]
	(No symbol) [0x00AB4A1F]
	(No symbol) [0x00ABA418]
	(No symbol) [0x00ABA505]
	(No symbol) [0x00AC508B]
	BaseThreadInitThunk [0x761B7D69+25]
	RtlInitializeExceptionChain [0x772BBB9B+107]
	RtlClearBits [0x772BBB1F+191]


In [79]:
# driver.find_element(By.CLASS_NAME,"HeaderMetadata__ReleaseDate-sc-1p42fnf-4 dEQuaq")

In [80]:
# reject all cookies
#driver.find_element(By.XPATH,"/html/body/div[6]/div[2]/div/div/div[2]/div/div/button[1]").click()

In [81]:
# for i in range(1,6): 
#     driver.implicitly_wait(2)
#     calendar = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]").click()
#     week_i = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[%d]/td[1]/span"%(i)).click()
#     date = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/h2").text

In [82]:
# calendar = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]").click()
# driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[1]/td[1]/span").click()
# date = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/h2").text
# date

In [83]:
# calendar = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]").click()
# previous_month = driver.find_element(By.XPATH,"/html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[1]/div[2]/button[1]").click()

In [84]:
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[4]/td[1] # s3
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[3]/td[1] # s2
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[2]/td[1] # xpath S1
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[5]/td[1] # s4 actuellement 
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[1]/td[1]/span
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[4]/td[1]/span
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/div/div[2]/table/tbody/tr[%d]/td[1]

In [85]:
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/h2
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/h2
# /html/body/div[4]/main/div[2]/div[1]/div/div/div[2]/div[2]/div[2]/div/div/h2